### Efficiency Analysis
These engine agnostic queries analyze spark history server task summaries and stages to surface inefficient spark applications based on skew, spill, partition size, processing rate, and task failure rates. The sink tables are optimized to allow for traditional sql based analytics or agentic natural language analysis as well. 

In [0]:
dbutils.widgets.text("catalog_name", "", "Catalog (required)")
dbutils.widgets.text("schema_name", "", "Schema")
CATALOG_NAME = dbutils.widgets.get("catalog_name").strip()
SCHEMA_NAME = dbutils.widgets.get("schema_name").strip() or "spark_observability"

# UC Validation
if not CATALOG_NAME:
    raise ValueError("catalog widget must point to an existing catalog")

spark.sql(f"CREATE SCHEMA IF NOT EXISTS {CATALOG_NAME}.{SCHEMA_NAME}")

In [0]:
 %sql
-- diff of more than 2x btwn min and median or median and max
-- any spill
-- chunk sizes
-- diff of more than 2x processing rate
-- with task completion rate
create or replace table IDENTIFIER(:catalog_name || '.' || :schema_name || '.' || 'ineffjobagg')
as
with rawmetrics as (
select *, try_variant_get(parse_json(raw_json),'$.duration', 'array<double>') as duration, try_variant_get(parse_json(raw_json),'$.memoryBytesSpilled', 'array<double>') as memoryBytesSpilled, try_variant_get(parse_json(raw_json),'$.diskBytesSpilled', 'array<double>') as diskBytesSpilled, try_variant_get(parse_json(raw_json),'$.inputMetrics.bytesRead', 'array<double>') as bytesRead , 
try_variant_get(parse_json(raw_json),'$.outputMetrics.bytesWritten', 'array<double>') as bytesWritten , try_variant_get(parse_json(raw_json),'$.shuffleReadMetrics.readBytes', 'array<double>') readBytes,
try_variant_get(parse_json(raw_json),'$.shuffleReadMetrics.fetchWaitTime', 'array<double>') fetchWaitTime,
try_variant_get(parse_json(raw_json),'$.shuffleWriteMetrics.writeBytes', 'array<double>') writeBytes
from prodrs.spark_observability.task_summaries),

polmetrics as (select *, duration[0] as min, duration[2] as median, duration[4] as maxim, memoryBytesSpilled[2] as spillmed, memoryBytesSpilled[4] as spillmax, diskBytesSpilled[2] as diskspillmed, diskBytesSpilled[4] as diskspillmax, bytesRead[0] as minbytes, bytesRead[2] as medbytes, bytesRead[4] as maxbytes, bytesWritten[0] as minbyteswr, bytesWritten[2] as medbyteswr, bytesWritten[4] as maxbyteswr, readbytes[0] as minsr, readbytes[2] as medsr, readbytes[4] as maxsr, writebytes[0] as minsw, writebytes[2] as medsw, writebytes[4] as maxsw,
map('minbytes', minbytes, 'minbyteswr', minbyteswr, 'minsr', minsr, 'minsw',minsw) as minchunks,
map('minbytes', medbytes, 'medbyteswr', medbyteswr, 'medsr', medsr, 'medsw',medsw) as medchunks,
map('maxbytes', maxbytes, 'maxbyteswr', maxbyteswr, 'maxsr', maxsr, 'maxsw',maxsw) as maxchunks,
map('minbrpr', try_divide(minbytes, min), 'minbwpr', try_divide(minbyteswr, min), 'minsrpr', try_divide(minsr, min), 'minswpr', try_divide(minsw, min)) as minpr,
map('medbrpr', try_divide(medbytes, median), 'medbwpr', try_divide(medbyteswr, median), 'medsrpr', try_divide(medsr, median), 'medswpr', try_divide(medsw, median)) as medpr,
map('maxbrpr', try_divide(maxbytes, maxim), 'maxbwpr', try_divide(maxbytes, maxim), 'maxsrpr', try_divide(maxsr, maxim), 'maxswpr', try_divide(maxsw, maxim)) as maxpr
from rawmetrics),

mapping as (select *, diskspillmed, diskspillmax,  map_filter(minchunks, (k, v) -> v > 512000000) as minchunkmap, map_filter(minchunks, (k, v) -> v > 512000000) as medchunkmap, map_filter(minchunks, (k, v) -> v > 512000000) as maxchunkmap,
map_filter(minpr, (k, v) -> v between 0.01 and  10000) as minprmap, map_filter(medpr, (k, v) -> v between 0.01 and 10000) as medprmap, map_filter(maxpr, (k, v) -> v between 0.01 and 10000) as maxprmap
from polmetrics),

precheck as (select cluster_id, cluster_name, stage_id, diskspillmed, diskspillmax, min, median, maxim, spillmed, spillmax, cardinality(minchunkmap) as minchunks, cardinality(medchunkmap) as medchunks, cardinality(maxchunkmap) as maxchunks,
cardinality(minprmap) as minpr, cardinality(medprmap) as medpr, cardinality(maxprmap) as maxpr
from mapping),

goldcheck as (select cluster_id, cluster_name, stage_id, CASE WHEN (median > (2 * min) OR maxim > (2 * median))  THEN 1 ELSE 0 END as skewbinary,
CASE WHEN (spillmed > 256000000 or spillmax > 256000000) THEN 1 ELSE 0 END as spillbinary,
CASE WHEN (diskspillmed > 256000000 or diskspillmax > 256000000) THEN 1 ELSE 0 END as diskspillbinary,
CASE WHEN (minchunks > 0 or medchunks > 0 or maxchunks > 0) THEN 1 ELSE 0 END as chunkbinary,
CASE WHEN (minpr > 0 or medpr > 0 or maxpr > 0) THEN 1 ELSE 0 END as prbinary
from precheck),

enrichjoin as (
  select goldcheck.*, cardinality(map_filter(map('skewbinary', skewbinary, 'spillbinary', spillbinary, 'diskspillbinary', diskspillbinary, 'chunkbinary', chunkbinary, 'trbinary', case when stages.task_completion_rate < 0.9 THEN 1 ELSE 0 END ), (k, v) -> v > 0)) as goldcheckfilter, stages.task_completion_rate, timestampdiff(millisecond, stages.submission_time, stages.completion_time) as duration
  from goldcheck
  join prodrs.spark_observability.stages as stages
  on goldcheck.cluster_id = stages.cluster_id
  and goldcheck.stage_id = stages.stage_id
),

pu as (select cluster_id, cluster_name, stage_id, sum(case when goldcheckfilter > 0 then duration else 0 end) as ineffstag,
sum(duration) as stageagg
from enrichjoin
group by all),

finalsa as (select cluster_id, cluster_name, try_divide(sum(ineffstag), sum(stageagg)) * 100 as ineffstageagg
from pu
group by all),

finaljoin as(
  select finalsa.*, apps.start_time
  from finalsa
  join prodrs.spark_observability.applications apps
  on finalsa.cluster_id = apps.cluster_id
)

select * from finaljoin
where ineffstageagg > 25
--order by 3 desc, 4 desc

In [0]:
 %sql
-- diff of more than 2x btwn min and median or median and max
-- any spill
-- chunk sizes
-- diff of more than 2x processing rate
-- with task completion rate

create or replace table IDENTIFIER(:catalog_name || '.' || :schema_name || '.' || 'ineffjobraw')
as
with rawmetrics as (
select *, try_variant_get(parse_json(raw_json),'$.duration', 'array<double>') as duration, try_variant_get(parse_json(raw_json),'$.memoryBytesSpilled', 'array<double>') as memoryBytesSpilled, try_variant_get(parse_json(raw_json),'$.diskBytesSpilled', 'array<double>') as diskBytesSpilled, try_variant_get(parse_json(raw_json),'$.inputMetrics.bytesRead', 'array<double>') as bytesRead , 
try_variant_get(parse_json(raw_json),'$.outputMetrics.bytesWritten', 'array<double>') as bytesWritten , try_variant_get(parse_json(raw_json),'$.shuffleReadMetrics.readBytes', 'array<double>') readBytes,
try_variant_get(parse_json(raw_json),'$.shuffleReadMetrics.fetchWaitTime', 'array<double>') fetchWaitTime,
try_variant_get(parse_json(raw_json),'$.shuffleWriteMetrics.writeBytes', 'array<double>') writeBytes
from prodrs.spark_observability.task_summaries),

polmetrics as (select *, duration[0] as min, duration[2] as median, duration[4] as maxim, memoryBytesSpilled[2] as spillmed, memoryBytesSpilled[4] as spillmax, diskBytesSpilled[2] as diskspillmed, diskBytesSpilled[4] as diskspillmax, bytesRead[0] as minbytes, bytesRead[2] as medbytes, bytesRead[4] as maxbytes, bytesWritten[0] as minbyteswr, bytesWritten[2] as medbyteswr, bytesWritten[4] as maxbyteswr, readbytes[0] as minsr, readbytes[2] as medsr, readbytes[4] as maxsr, writebytes[0] as minsw, writebytes[2] as medsw, writebytes[4] as maxsw,
map('minbytes', minbytes, 'minbyteswr', minbyteswr, 'minsr', minsr, 'minsw',minsw) as minchunks,
map('minbytes', medbytes, 'medbyteswr', medbyteswr, 'medsr', medsr, 'medsw',medsw) as medchunks,
map('maxbytes', maxbytes, 'maxbyteswr', maxbyteswr, 'maxsr', maxsr, 'maxsw',maxsw) as maxchunks,
map('minbrpr', try_divide(minbytes, min), 'minbwpr', try_divide(minbyteswr, min), 'minsrpr', try_divide(minsr, min), 'minswpr', try_divide(minsw, min)) as minpr,
map('medbrpr', try_divide(medbytes, median), 'medbwpr', try_divide(medbyteswr, median), 'medsrpr', try_divide(medsr, median), 'medswpr', try_divide(medsw, median)) as medpr,
map('maxbrpr', try_divide(maxbytes, maxim), 'maxbwpr', try_divide(maxbytes, maxim), 'maxsrpr', try_divide(maxsr, maxim), 'maxswpr', try_divide(maxsw, maxim)) as maxpr
from rawmetrics),

mapping as (select *, diskspillmed, diskspillmax,  map_filter(minchunks, (k, v) -> v > 512000000) as minchunkmap, map_filter(minchunks, (k, v) -> v > 512000000) as medchunkmap, map_filter(minchunks, (k, v) -> v > 512000000) as maxchunkmap,
map_filter(minpr, (k, v) -> v between 0.01 and  10000) as minprmap, map_filter(medpr, (k, v) -> v between 0.01 and 10000) as medprmap, map_filter(maxpr, (k, v) -> v between 0.01 and 10000) as maxprmap
from polmetrics),

precheck as (select cluster_id, cluster_name, stage_id, diskspillmed, diskspillmax, min, median, maxim, spillmed, spillmax, cardinality(minchunkmap) as minchunks, cardinality(medchunkmap) as medchunks, cardinality(maxchunkmap) as maxchunks,
cardinality(minprmap) as minpr, cardinality(medprmap) as medpr, cardinality(maxprmap) as maxpr
from mapping),

goldcheck as (select cluster_id, cluster_name, stage_id, CASE WHEN (median > (2 * min) OR maxim > (2 * median))  THEN 1 ELSE 0 END as skewbinary,
CASE WHEN (spillmed > 256000000 or spillmax > 256000000) THEN 1 ELSE 0 END as spillbinary,
CASE WHEN (diskspillmed > 256000000 or diskspillmax > 256000000) THEN 1 ELSE 0 END as diskspillbinary,
CASE WHEN (minchunks > 0 or medchunks > 0 or maxchunks > 0) THEN 1 ELSE 0 END as chunkbinary,
CASE WHEN (minpr > 0 or medpr > 0 or maxpr > 0) THEN 1 ELSE 0 END as prbinary
from precheck),

enrichjoin as (
  select goldcheck.*, cardinality(map_filter(map('skewbinary', skewbinary, 'spillbinary', spillbinary, 'diskspillbinary', diskspillbinary, 'chunkbinary', chunkbinary, 'trbinary', case when stages.task_completion_rate < 0.9 THEN 1 ELSE 0 END ), (k, v) -> v > 0)) as goldcheckfilter, stages.task_completion_rate, timestampdiff(millisecond, stages.submission_time, stages.completion_time) as duration, stages.completion_time
  from goldcheck
  join prodrs.spark_observability.stages as stages
  on goldcheck.cluster_id = stages.cluster_id
  and goldcheck.stage_id = stages.stage_id
)

select * from enrichjoin
--where cluster_id = '1108-063956-zfgrta8i'
--where ineffstageagg > 25
--order by 3 desc, 4 desc